In [6]:
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [14]:
# --- Step 1: Create sample data ---
texts = [
    "Milvus is a vector database designed for scalable similarity search.",
    "Retrieval-Augmented Generation combines retrieval with generation.",
    "LangChain provides tools for building RAG pipelines easily.",
    "Milvus is so cool!"
]

# --- Step 2: Create embeddings ---
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# --- Step 3: Store embeddings in Milvus ---
vector_store = Milvus.from_texts(
    texts,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="rag_docs"
)


In [15]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Retrieve top 2 most similar documents for a query
query = "What is Milvus used for?"
results = retriever.get_relevant_documents(query)

# Display retrieved texts
for i, doc in enumerate(results):
    print(f"\n--- Retrieved Document {i+1} ---")
    print(doc.page_content)
    
    print("Metadata:", doc.metadata)




--- Retrieved Document 1 ---
Milvus is so cool!
Metadata: {'pk': 461544318759744073}

--- Retrieved Document 2 ---
Milvus is a vector database designed for scalable similarity search.
Metadata: {'pk': 461544318759744061}


In [16]:

# # --- Step 4: Create retriever ---
# retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# --- Step 5: Connect to LLM ---
llm = ChatOpenAI(model="gpt-5-mini")


In [17]:

# --- Step 6: Build RAG QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # or "map_reduce"
)

# --- Step 7: Ask a question ---
query = "What is Milvus used for?"
response = qa_chain.run(query)
print(response)

Milvus is a vector database used to store, index and search high‑dimensional vectors (embeddings) for fast similarity search. In practice that means you use it to power applications that need to find “things that are similar” at scale.

Typical uses
- Semantic / full‑text search (retrieve passages or documents by meaning, not exact keywords)
- Recommendation and personalization (find similar users/items)
- Image, video and face search (find visually similar items)
- Audio / speech search and retrieval
- Question‑answering and knowledge retrieval (retrieve relevant contexts for LLMs)
- Anomaly detection (find outliers in embedding space)

How it’s used (high level)
- Create embeddings from data with an ML model (text/image/audio → vector)
- Store vectors in Milvus and build an index
- Run nearest‑neighbor queries to retrieve similar vectors quickly
- Post‑process results (filter, rank, display, feed to downstream models)

Key advantages
- Optimized for large-scale similarity search (bil